In [1]:
!pip install tensorflow_datasets
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, Bidirectional, LSTM, Dense, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
print(tf.__version__)

2.20.0


In [2]:

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
# 1. Caricamento dei dati
train_df = pd.read_csv(train_file_path, sep='\t', names=['label', 'text'])
test_df = pd.read_csv(test_file_path, sep='\t', names=['label', 'text'])

# 2. Conversione etichette in numeriche (ham=0, spam=1)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Parametri per la riproducibilità e scalabilità
MAX_FEATURES = 10000  # Dimensione del vocabolario
SEQUENCE_LENGTH = 250 # Lunghezza massima dell'SMS
BATCH_SIZE = 32
# Creazione del layer di vettorizzazione
vectorizer = TextVectorization(
    max_tokens=MAX_FEATURES,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH,
)

# Adattamento del vocabolario ai dati di training
vectorizer.adapt(train_df['text'].values)

# Creazione dei dataset TensorFlow per performance ottimali
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df['text'].values, train_df['label'].values)
).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df['text'].values, test_df['label'].values)
).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [4]:
model = Sequential([
    Input(shape=(1,), dtype=tf.string),
    vectorizer,
    Embedding(MAX_FEATURES, 64, mask_zero=True),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dense(64, activation='relu'),
    Dropout(0.5), # Previene l'overfitting, fondamentale per il CV
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=6, 
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=2, 
    min_lr=0.00001,
    verbose=1
)

# Training rapido ma efficace
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9301 - loss: 0.2394 - precision: 0.8895 - recall: 0.5464 - val_accuracy: 0.9820 - val_loss: 0.0533 - val_precision: 0.9602 - val_recall: 0.9037 - learning_rate: 0.0010
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.9931 - loss: 0.0306 - precision: 0.9819 - recall: 0.9661 - val_accuracy: 0.9878 - val_loss: 0.0376 - val_precision: 0.9775 - val_recall: 0.9305 - learning_rate: 0.0010
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.9983 - loss: 0.0083 - precision: 0.9982 - recall: 0.9893 - val_accuracy: 0.9885 - val_loss: 0.0388 - val_precision: 0.9777 - val_recall: 0.9358 - learning_rate: 0.0010
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.9995 - loss: 0.0024 - precision: 1.0000 - recall: 0.9964 - val_accuracy: 0.9885 - val_loss: 0.0363 - val_precision: 0.9572 - val_recall: 0.9572 - learning_rate: 0.0010
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accur

In [5]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # 1. Preparazione dell'input come tensore di stringhe
    # Avvolgiamo il testo in una lista e poi in un array per creare il batch
    input_tensor = tf.constant([pred_text])
    
    # 2. Inferenza diretta
    # Chiamare il modello come una funzione è più veloce e flessibile di .predict()
    # per una singola stringa.
    prediction = model(input_tensor, training=False)
    
    # 3. Estrazione del valore
    proba = float(prediction[0][0])
    
    # 4. Logica di classificazione
    label = "spam" if proba >= 0.5 else "ham"
    
    return [proba, label]

# Test manuale
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[3.023942008439917e-05, 'ham']
[3.023942008439917e-05, 'ham']


In [6]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
